In [1]:
from transformers import pipeline

checkpoint = "openai/clip-vit-large-patch14"
detector = pipeline(model=checkpoint, task="zero-shot-image-classification")

config.json:   0%|          | 0.00/4.52k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/905 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/961k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.50, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Device set to use cuda:0


In [2]:
from datasets import load_dataset

dataset = load_dataset("imagefolder", data_dir="daten")
dataset

Resolving data files:   0%|          | 0/3115 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/351 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/359 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 3115
    })
    validation: Dataset({
        features: ['image', 'label'],
        num_rows: 351
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 359
    })
})

In [3]:
from PIL import Image
from tqdm import tqdm

labels_fruit_and_vegetable = ['apple', 'banana', 'beetroot', 'bell pepper', 'cabbage', 'capsicum', 'carrot', 'cauliflower', 'chilli pepper', 'corn', 'cucumber', 'eggplant', 'garlic', 'ginger', 'grapes', 'jalepeno', 'kiwi', 'lemon', 'lettuce', 'mango', 'onion', 'orange', 'paprika', 'pear', 'peas', 'pineapple', 'pomegranate', 'potato', 'raddish', 'soy beans', 'spinach', 'sweetcorn', 'sweetpotato', 'tomato', 'turnip', 'watermelon']

true_labels = []
predicted_labels = []

In [4]:
labels = dataset["train"].features["label"].names
label2id = {label: i for i, label in enumerate(labels)}
id2label = {i: label for i, label in enumerate(labels)}

In [6]:
from PIL import Image
from PIL import ImageFile
import os
from pathlib import Path

ImageFile.LOAD_TRUNCATED_IMAGES = True

def check_images_in_directory(directory):
    # Überprüfen, ob das Verzeichnis existiert
    directory = Path(directory)
    
    if not directory.exists():
        print(f"Das Verzeichnis {directory} wurde nicht gefunden!")
        return

    # Durchläuft alle Dateien und Unterordner im angegebenen Verzeichnis
    for file_path in directory.rglob("*"):  # rglob geht rekursiv durch alle Unterordner
        if file_path.suffix in [".jpg", ".png"]:  # Überprüft, ob es eine .jpg oder .png Datei ist
            try:
                img = Image.open(file_path)
                img.verify()  # Überprüft, ob das Bild gültig ist
            except (IOError, SyntaxError) as e:
                print(f"Fehler beim Laden des Bildes: {file_path}")

# Beispielverzeichnis auf lightning.ai (Anpassen je nach Speicherort)
check_images_in_directory("daten")

In [7]:
for i in tqdm(range(len(dataset["train"]))):
    # Bild direkt aus dem Dataset holen (ist bereits ein PIL.Image)
    image = dataset["train"][i]["image"]

    results = detector(image, candidate_labels=labels_fruit_and_vegetable)

    top_prediction = max(results, key=lambda x: x["score"])["label"]

    true_label = id2label[dataset["train"][i]["label"]]

    true_labels.append(true_label)
    predicted_labels.append(top_prediction)

100%|██████████| 3115/3115 [07:34<00:00,  6.86it/s]


In [8]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

# Calculate accuracy
accuracy = accuracy_score(true_labels, predicted_labels)

# Calculate precision and recall
precision = precision_score(true_labels, predicted_labels, average='weighted', labels=labels_fruit_and_vegetable)
recall = recall_score(true_labels, predicted_labels, average='weighted', labels=labels_fruit_and_vegetable)

# Print the results
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")

Accuracy: 0.8815
Precision: 0.8983
Recall: 0.8815


In [1]:
import gradio as gr
from transformers import pipeline

vit_classifier = pipeline("image-classification", model="ElioBaserga/fruits-and-vegetables-vit")
clip_detector = pipeline(model="openai/clip-vit-large-patch14", task="zero-shot-image-classification")

labels_fruit_and_vegetable = ['apple', 'banana', 'beetroot', 'bell pepper', 'cabbage', 'capsicum', 'carrot', 'cauliflower', 'chilli pepper', 'corn', 'cucumber', 'eggplant', 'garlic', 'ginger', 'grapes', 'jalepeno', 'kiwi', 'lemon', 'lettuce', 'mango', 'onion', 'orange', 'paprika', 'pear', 'peas', 'pineapple', 'pomegranate', 'potato', 'raddish', 'soy beans', 'spinach', 'sweetcorn', 'sweetpotato', 'tomato', 'turnip', 'watermelon']

def classify_pet(image):
    vit_results = vit_classifier(image)
    vit_output = {result['label']: result['score'] for result in vit_results}
    
    clip_results = clip_detector(image, candidate_labels=labels_fruit_and_vegetable)
    clip_output = {result['label']: result['score'] for result in clip_results}
    
    return {"ViT Classification": vit_output, "CLIP Zero-Shot Classification": clip_output}

example_images = [
    ["daten/daten/test/apple/Image_1.jpg"],
    ["daten/daten/test/cauliflower/Image_3.jpg"],
    ["daten/daten/test/kiwi/Image_6.jpg"],
    ["daten/daten/test/peas/Image_9.jpg"],
    ["daten/daten/test/onion/Image_5.jpg"]
]

iface = gr.Interface(
    fn=classify_pet,
    inputs=gr.Image(type="filepath"),
    outputs=gr.JSON(),
    title="Fruit and Vegetable Classification Comparison",
    description="Upload an image of a fruit or vegetable, and compare results from a trained ViT model and a zero-shot CLIP model.",
    examples=example_images
)

iface.launch(share=True)

config.json:   0%|          | 0.00/2.09k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/343M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/351 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.50, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Device set to use cpu


config.json:   0%|          | 0.00/4.52k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

Device set to use cpu


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://6287684715d18441e6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
